In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

Question 1

(a)

In [ ]:
def f(ls):
    x = []
    for i in ls:
        x.append(str(i))
    return x

In [18]:
n = 1000
def option_pricing(n, K):
    r = .05
    sigma = .1
    S0 = 100
    T = 1
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    z = np.random.normal(0, 1, n)
    St = S0 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * z)
    Ct = St - K
    Ct[Ct < 0] = 0
    c = discFactor * Ct
    return (c.mean(), c.std() / np.sqrt(n))


Ks = [95, 100, 105]

pricing = np.array([list(option_pricing(1000, i)) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]},
                     index=Ks)


result

,Price,Std
95,10.808553,0.287336
100,6.865759,0.248153
105,4.159802,0.203730


(b)

In [31]:
def option_pricing_anti(n, K):
    r = .05
    sigma = .1
    S0 = 100
    T = 1
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    U = np.random.uniform(0, 1, n)
    Ua = 1 - U
    Z = norm.ppf(U)
    Za = norm.ppf(1 - U)
    St = S0 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)
    Ct = St - K
    Ct[Ct < 0] = 0
    Sta = S0 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * Za)
    Cta = Sta - K
    Cta[Cta < 0] = 0
    c = discFactor * (Ct + Cta) / 2
    return (c.mean(), c.std() / np.sqrt(n))


Ks = [95, 100, 105]

pricing = np.array([list(option_pricing_anti(1000, i)) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]},
                     index=Ks)

result  

,Price,Std
95,10.342667,0.069359
100,6.703201,0.090125
105,4.180335,0.104638


Question 2

(a)

In [37]:
n = 1000
def Asian_option_pricing(n, periods, K):
    r = .05
    sigma = .1
    S0 = 100
    T = 1
    delta = T / float(periods)
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    z = np.random.normal(0, 1, (n, periods))
    prices_paths = np.zeros((n, periods))
    S1 = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z[:, 0])
    prices_paths[:, 0] = S1
    for i in xrange(1, periods):
        prices_paths[:, i] = prices_paths[:, i - 1] *\
                                np.exp((r - .5 * sigma ** 2) * delta +
                                       sigma * np.sqrt(delta) * z[:, i])

    Ct = prices_paths.mean(axis=1) - K
    Ct[Ct < 0] = 0
    c = discFactor * Ct
    return (c.mean(), c.std() / np.sqrt(n))


Ks = [95, 100, 105]

pricing = np.array([list(Asian_option_pricing(1000, 52, i)) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]},
                     index=Ks)

result  

,Price,Std
95,7.549784,0.172068
100,3.658434,0.133070
105,1.293816,0.084255


(b)

In [38]:
n = 1000
def Asian_option_pricing_anti(n, periods, K):
    r = .05
    sigma = .1
    S0 = 100
    T = 1
    delta = T / float(periods)
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    z = np.random.normal(0, 1, (n, periods))
    za = -z
    prices_paths = np.zeros((n, periods))
    prices_paths_a = np.zeros((n, periods))
    S1 = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z[:, 0])
    S1a = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * za[:, 0])
    prices_paths[:, 0] = S1
    prices_paths_a[:, 0] = S1a
    for i in xrange(1, periods):
        prices_paths[:, i] = prices_paths[:, i - 1] *\
                                np.exp((r - .5 * sigma ** 2) * delta +
                                       sigma * np.sqrt(delta) * z[:, i])

        prices_paths_a[:, i] = prices_paths_a[:, i - 1] *\
                                np.exp((r - .5 * sigma ** 2) * delta +
                                       sigma * np.sqrt(delta) * za[:, i])

    Ct = prices_paths.mean(axis=1) - K
    Ct[Ct < 0] = 0
    Cta = prices_paths_a.mean(axis=1) - K
    Cta[Cta < 0] = 0
    c = discFactor * (Ct + Cta) / 2
    return (c.mean(), c.std() / np.sqrt(n))


Ks = [95, 100, 105]

pricing = np.array([list(Asian_option_pricing_anti(1000, 52, i)) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]},
                     index=Ks)

result

,Price,Std
95,7.443312,0.028142
100,3.673376,0.052394
105,1.383220,0.052350


(c)

In [54]:
n = 1000
def Asian_option_pricing_control_variable(n, periods, K):
    r = .05
    sigma = .1
    S0 = 100
    T = 1
    delta = T / float(periods)
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    z = np.random.normal(0, 1, (n, periods))
    prices_paths = np.zeros((n, periods))
    S1 = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z[:, 0])
    prices_paths[:, 0] = S1
    for i in xrange(1, periods):
        prices_paths[:, i] = prices_paths[:, i - 1] *\
                                np.exp((r - .5 * sigma ** 2) * delta +
                                       sigma * np.sqrt(delta) * z[:, i])
    
    Y = (prices_paths.mean(axis=1) - K) * discFactor
    Y[Y < 0] = 0
    X = prices_paths[:, -1]
    sig_x = np.sum((X - X.mean()) ** 2 / (len(X) - 1)) ** 0.5
    sig_xy = np.cov(X, Y)[0, 1] * len(Y) / (len(Y) - 1)
    a_hat = -sig_xy / sig_x ** 2
    c = Y + a_hat * (X - S0 / discFactor)
    
#     c = discFactor * Ct
    return (c.mean(), c.std() / np.sqrt(n))


Ks = [95, 100, 105]

pricing = np.array([list(Asian_option_pricing_control_variable(1000, 52, i)) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]},
                     index=Ks)

result 

,Price,Std
95,7.590567,0.085901
100,3.625191,0.080949
105,1.346843,0.059987


(d)

In [69]:
n = 1000

# def d1(S0, K, r, sigma, T):
#     return (np.log(S0 / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
 
# def d2(S0, K, r, sigma, T):
#     return (np.log(S0 / K) + (r - sigma**2 / 2) * T) / (sigma * np.sqrt(T))
 
# def BlackScholes(type,S0, K, r, sigma, T):
#     if type=="C":
#         return S0 * ss.norm.cdf(d1(S0, K, r, sigma, T)) - K * np.exp(-r * T) * norm.cdf(d2(S0, K, r, sigma, T))
#     else:
#         return K * np.exp(-r * T) * ss.norm.cdf(-d2(S0, K, r, sigma, T)) - S0 * norm.cdf(-d1(S0, K, r, sigma, T))

def black_scholes (cp, s, k, t, v, rf, div):
#         """ Price an option using the Black-Scholes model.
#         s: initial stock price
#         k: strike price
#         t: expiration time
#         v: volatility
#         rf: risk-free rate
#         div: dividend
#         cp:  + 1 / -1 for call / put
#         """
        d1 = (np.log(s / k) + (rf - div + 0.5 * (v ** 2) * t)) / (v * np.sqrt(t))
        d2 = d1 - v * np.sqrt(t)

        optprice = (cp * s * np.exp(-div * t) *
                    norm.cdf(cp * d1)) - (cp * k * np.exp(-rf * t) *
                                          norm.cdf(cp * d2))
        return optprice


def Asian_option_BS(K):
    r = .05
    sigma = .1
    S0 = 100.0
    T = 1.0
    sigma_ = sigma / np.sqrt(3)
    d_ = .5 * (r + (sigma ** 2) / 6)
    return black_scholes(1, S0, K, T, sigma_, r, d_)
 
def Asian_option_pricing_control_variable2(n, periods, K):
    r = .05
    sigma = .1
    S0 = 100
    T = 1
    delta = T / float(periods)
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    z = np.random.normal(0, 1, (n, periods))
    prices_paths = np.zeros((n, periods))
    S1 = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z[:, 0])
    prices_paths[:, 0] = S1
    X_mean = Asian_option_BS(K)
    for i in xrange(1, periods):
        prices_paths[:, i] = prices_paths[:, i - 1] *\
                                np.exp((r - .5 * sigma ** 2) * delta +
                                       sigma * np.sqrt(delta) * z[:, i])
    
    Y = (prices_paths.mean(axis=1) - K) * discFactor
    Y[Y < 0] = 0
    X = ((np.multiply.reduce(prices_paths, axis=1)) ** delta - K) * discFactor
    X[X < 0] = 0
    sig_x = np.sum((X - X.mean()) ** 2 / (len(X) - 1)) ** 0.5
    sig_xy = np.cov(X, Y)[0, 1] * len(Y) / (len(Y) - 1)
    a_hat = -sig_xy / sig_x ** 2
    c = Y + a_hat * (X - X_mean)
    
#     c = discFactor * Ct
    return (c.mean(), c.std() / np.sqrt(n))


Ks = [95, 100, 105]

pricing = np.array([list(Asian_option_pricing_control_variable2(1000, 52, i)) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]},
                     index=Ks)

result 

,Price,Std
95,7.406771,0.002261
100,3.640158,0.001928
105,1.310307,0.001810


(e)

In [70]:
n = 1000

def black_scholes (cp, s, k, t, v, rf, div):
#         """ Price an option using the Black-Scholes model.
#         s: initial stock price
#         k: strike price
#         t: expiration time
#         v: volatility
#         rf: risk-free rate
#         div: dividend
#         cp:  + 1 / -1 for call / put
#         """
        d1 = (np.log(s / k) + (rf - div + 0.5 * (v ** 2) * t)) / (v * np.sqrt(t))
        d2 = d1 - v * np.sqrt(t)

        optprice = (cp * s * np.exp(-div * t) *
                    norm.cdf(cp * d1)) - (cp * k * np.exp(-rf * t) *
                                          norm.cdf(cp * d2))
        return optprice


def Asian_option_BS2(K, N):
    N = float(N)
    r = .05
    sigma = .1
    S0 = 100.0
    T = 1.0
    sigma_ = sigma * np.sqrt((N + 1) * (2 * N + 1) / (6 * N ** 2))
    d_ = r * (N - 1) / (2 * N) + sigma ** 2 * ((N + 1) * (N - 1) / (12 * N ** 12))
#     d_ = .5 * (r + (sigma ** 2) / 6)
    return black_scholes(1, S0, K, T, sigma_, r, d_)
 
def Asian_option_pricing_control_variable3(n, periods, K):
    r = .05
    sigma = .1
    S0 = 100
    T = 1
    delta = T / float(periods)
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    z = np.random.normal(0, 1, (n, periods))
    prices_paths = np.zeros((n, periods))
    S1 = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z[:, 0])
    prices_paths[:, 0] = S1
    X_mean = Asian_option_BS2(K, periods)
    for i in xrange(1, periods):
        prices_paths[:, i] = prices_paths[:, i - 1] *\
                                np.exp((r - .5 * sigma ** 2) * delta +
                                       sigma * np.sqrt(delta) * z[:, i])
    
    Y = (prices_paths.mean(axis=1) - K) * discFactor
    Y[Y < 0] = 0
    X = ((np.multiply.reduce(prices_paths, axis=1)) ** delta - K) * discFactor
    X[X < 0] = 0
    sig_x = np.sum((X - X.mean()) ** 2 / (len(X) - 1)) ** 0.5
    sig_xy = np.cov(X, Y)[0, 1] * len(Y) / (len(Y) - 1)
    a_hat = -sig_xy / sig_x ** 2
    c = Y + a_hat * (X - X_mean)
    return (c.mean(), c.std() / np.sqrt(n))


Ks = [95, 100, 105]

pricing = np.array([list(Asian_option_pricing_control_variable3(1000, 52, i)) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]},
                     index=Ks)

result 

,Price,Std
95,7.540045,0.002084
100,3.759879,0.001981
105,1.384728,0.001838


Question 3

(a)

In [74]:
n = 10000

def option_pricing(n, K):
    r = .05
    sigma = .2
    S0 = 100
    T = 1
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    z = np.random.normal(0, 1, n)
    St = S0 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * z)
    Ct = St - K
    Ct[Ct < 0] = 0
    c = discFactor * Ct
    return (c.mean(), c.std() / np.sqrt(n))


Ks = [120, 140, 160]

pricing = np.array([list(option_pricing(n, i)) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]},
                     index=Ks)


result

,Price,Std
120,3.169959,0.085269
140,0.722872,0.039585
160,0.157853,0.018783


(b)

In [121]:
n = 10000

def option_pricing_putcall(n, K):
    r = .05
    sigma = .2
    S0 = 100
    T = 1
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    z = np.random.normal(0, 1, n)
    St = S0 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * z)
    Pt = K - St
    Pt[Pt < 0] = 0
    p = discFactor * Pt
    Ct = p + S0 - K * np.exp(-r * T)
    c = Ct
    return (c.mean(), c.std() / np.sqrt(n))


Ks = [120, 140, 160]

pricing = np.array([list(option_pricing_putcall(n, i)) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]},
                     index=Ks)


result

,Price,Std
120,3.112443,0.148002
140,1.033429,0.184284
160,0.140584,0.197962


(c)

In [138]:
n = 10000

def option_pricing_putcall_control(n, K):
    r = .05
    sigma = .2
    S0 = 100
    T = 1
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    z = np.random.normal(0, 1, n)
    St = S0 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * z)
    Pt = K - St
    Pt[Pt < 0] = 0
    p = discFactor * Pt
    Y = p
    X = St
    sig_x = np.sum((X - X.mean()) ** 2 / (len(X) - 1)) ** 0.5
    sig_xy = np.cov(X, Y)[0, 1] * len(Y) / (len(Y) - 1)
    a_hat = -sig_xy / sig_x ** 2
    p0 = Y + a_hat * (X - S0 / discFactor)
    c = p0 + S0 - K * np.exp(-r * T)
    return (c.mean(), c.std() / np.sqrt(n))


Ks = [120, 140, 160]

pricing = np.array([list(option_pricing_putcall_control(n, i)) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]},
                     index=Ks)


result

,Price,Std
120,3.283209,0.057810
140,0.792459,0.035888
160,0.169561,0.020039


(d)

In [120]:
n = 10000

def importance_sample(n, K):
    K = float(K)
    S0 = 100
    r = .05
    sigma = .2
    T = 1
    L = (np.log(K / S0) - ((r - 0.5 * (sigma ** 2)) * T)) / (sigma * np.sqrt(T))
    threhd = norm.cdf(L)
    U = np.random.uniform(0, 1, n)
    X = norm.ppf(U * (1 - threhd) + threhd)
    return (L, X)


def option_pricing_importance(n, K):
    r = .05
    sigma = .2
    S0 = 100
    T = 1
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    (L, z) = importance_sample(n, K)
    St = S0 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * z)
    Ct = St - K
    c = discFactor * Ct * (1 - norm.cdf(L))
    return (c.mean(), c.std() / np.sqrt(n))


Ks = [120, 140, 160]
ix = f(Ks)

pricing = np.array([list(option_pricing_importance(n, i)) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]},
                     index=Ks)


result

,Price,Std
120,3.359431,0.030936
140,0.790426,0.007501
160,0.160357,0.001568


Question 4

(a)

In [30]:
n = 100000
def digital_option_pricing(n, periods, K, H):
    r = .05
    sigma = .15
    S0 = 95
    T = 0.25
    delta = T / float(periods)
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    z = np.random.normal(0, 1, (n, periods))
    prices_paths = np.zeros((n, periods))
    S1 = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z[:, 0])
    prices_paths[:, 0] = S1
    for i in xrange(1, periods):
        prices_paths[:, i] = prices_paths[:, i - 1] *\
                                np.exp((r - .5 * sigma ** 2) * delta +
                                       sigma * np.sqrt(delta) * z[:, i])
    
    knocin_pos = (prices_paths < H).sum(axis=1) > 0
    prices_in = prices_paths[:, -1] > K
    Ct = np.zeros(n)
    Ct[knocin_pos & prices_in] = 10000
    c = discFactor * Ct
    return (c.mean(), c.std() / np.sqrt(n))


Ks = [[94, 96], [90, 96], [85, 96], [90, 106]]
Ks = np.array(Ks)
# ix = np.apply_along_axis(lambda x: str(x), 1, Ks)
ix = f(Ks)
# print ix
pricing = np.array([list(digital_option_pricing(100000, 50,
                                                i[1], i[0])) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]},
                     index=ix)

result  

,Price,Std
[94 96],3018.926578,14.387610
[90 96],425.942305,6.344356
[85 96],6.518013,0.802047
[ 90 106],15.109940,1.220631


In [37]:
n = 100000
def digital_option_pricing(n, periods, K, H):
    r = .05
    sigma = .15
    S0 = 95
    T = 1
    delta = T / float(periods)
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    z = np.random.normal(0, 1, (n, periods))
    prices_paths = np.zeros((n, periods))
    S1 = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z[:, 0])
    prices_paths[:, 0] = S1
    for i in xrange(1, periods):
        prices_paths[:, i] = prices_paths[:, i - 1] *\
                                np.exp((r - .5 * sigma ** 2) * delta +
                                       sigma * np.sqrt(delta) * z[:, i])
    
    knocin_pos = (prices_paths < H).sum(axis=1) > 0
    prices_in = prices_paths[:, -1] > K
    Ct = np.zeros(n)
    Ct[knocin_pos & prices_in] = 10000
    c = discFactor * Ct
    return (c.mean(), c.std() / np.sqrt(n))


Ks = np.array([(90, 106), (85, 96)])
# Ks =[[94, 96]]
ix = np.apply_along_axis(lambda x: str(x), 1, Ks)

pricing = np.array([list(digital_option_pricing(100000, 50,
                                                i[1], i[0])) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]},
                     index=ix)

result  

,Price,Std
[ 90 106],654.540967,7.614304
[85 96],448.029059,6.372640


In [21]:
n = 100000
def digital_option_pricing(n, periods, K, H):
    r = .05
    sigma = .15
    S0 = 95
    T = 0.25
    delta = T / float(periods)
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    z = np.random.normal(0, 1, (n, periods))
    prices_paths = np.zeros((n, periods))
    S1 = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z[:, 0])
    prices_paths[:, 0] = S1
    for i in xrange(1, periods):
        prices_paths[:, i] = prices_paths[:, i - 1] *\
                                np.exp((r - .5 * sigma ** 2) * delta +
                                       sigma * np.sqrt(delta) * z[:, i])
    
    knocin_pos = (prices_paths < H).sum(axis=1) > 0
    prices_in = prices_paths[:, -1] > K
    Ct = np.zeros(n)
    Ct[knocin_pos & prices_in] = 10000
    c = discFactor * Ct
    return (c.mean(), c.std() / np.sqrt(n))


# Ks = np.array([(90, 106), (85, 96)])
Ks = [[96,106], [85,96]]
def f(ls):
    x = []
    for i in ls:
        x.append(str(i))
    return x
ix = f(Ks)
print ix
pricing = np.array([list(digital_option_pricing(100000, 100,
                                                i[1], i[0])) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]},
                     index=ix)

result  

['[96, 106]', '[85, 96]']


,Price,Std
"[96, 106]",865.414427,8.830458
"[85, 96]",6.221740,0.783619


(b)

In [23]:
n = 100000


def digital_option_helper(K, St, r, tao, sigma):
    K = float(K)
    discFactor = np.exp(-r * tao)
#     print "hello"
    if St == 0:
        print "1"
    prob = norm.cdf((np.log(K / St) - (r - 0.5 * sigma ** 2) * tao) / (sigma * tao ** 0.5))
    return discFactor * (1 - prob)

def find_inital_time(price_path, H):
    judge = np.where(price_path < H)[0]
    if len(judge) > 0:
        return judge[0]
    else:
        return -1


def get_prices(sample_matrix, inital_time, delta, T, K, sigma, r):
    paras = []
    c = np.zeros(len(inital_time))

    for i in xrange(len(inital_time)):
        if inital_time[i] != -1:
            t = (inital_time[i] + 1) * delta
            tao = T - t
            if tao == 0:
                c[i] = 0
            else:
                St = sample_matrix[i, inital_time[i]]
                discfactor = np.exp(-r * t)
                c[i] = digital_option_helper(K, St, r, tao, sigma) * discfactor * 10000
    return c
    
        
def digital_option_pricing(n, periods, K, H):
    r = .05
    sigma = .15
    S0 = 95
    T = 0.25
    delta = T / float(periods)
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    z = np.random.normal(0, 1, (n, periods))
    prices_paths = np.zeros((n, periods))
    S1 = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z[:, 0])
    prices_paths[:, 0] = S1
    for i in xrange(1, periods):
        prices_paths[:, i] = prices_paths[:, i - 1] *\
                                np.exp((r - .5 * sigma ** 2) * delta +
                                       sigma * np.sqrt(delta) * z[:, i])
    initial_times = np.apply_along_axis(find_inital_time, 1, prices_paths, H)
#     print initial_times
    c = get_prices(prices_paths, initial_times, delta, T, K, sigma, r)
#     knocin_pos = (prices_paths < H).sum(axis=1) > 0
#     prices_in = prices_paths[:, -1] > K
#     Ct = np.zeros(n)
#     Ct[knocin_pos & prices_in] = 10000
#     c = discFactor * Ct
    return (c.mean(), c.std() / np.sqrt(n))


Ks = np.array([[94, 96], [90, 96], [85, 96], [90, 106]])
# ix = np.apply_along_axis(lambda x: str(x), 0, Ks)
def f(ls):
    x = []
    for i in ls:
        x.append(str(i))
    return x
ix = f(Ks)
print ix
pricing = np.array([list(digital_option_pricing(100000, 50,
                                                i[1], i[0])) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]}, index=ix)

result  

['[94 96]', '[90 96]', '[85 96]', '[ 90 106]']


,Price,Std
[94 96],3014.372508,5.063318
[90 96],419.933437,2.093601
[85 96],5.490133,0.098978
[ 90 106],13.335477,0.092661


(c)

In [24]:
n = 100000

def digital_option_helper(K, St, r, tao, sigma):
    K = float(K)
    discFactor = np.exp(-r * tao)
#     print "hello"
    if St == 0:
        print "1"
    prob = norm.cdf((np.log(K / St) - (r - 0.5 * sigma ** 2) * tao) / (sigma * tao ** 0.5))
    return discFactor * (1 - prob)

def find_inital_time(price_path, H):
    judge = np.where(price_path < H)[0]
    if len(judge) > 0:
        return judge[0]
    else:
        return -1


def get_prices(sample_matrix, inital_time, delta, T, K, sigma, r):
    paras = []
    c = np.zeros(len(inital_time))

    for i in xrange(len(inital_time)):
        if inital_time[i] != -1:
            t = (inital_time[i] + 1) * delta
            tao = T - t
            if tao == 0:
                c[i] = 0
            else:
                St = sample_matrix[i, inital_time[i]]
                discfactor = np.exp(-r * t)
                c[i] = digital_option_helper(K, St, r, tao, sigma) * discfactor * 10000
    return c


def digital_option_pricing_bthree(n, periods, K, H):
    r = .05
    sigma = .15
    S0 = 95
    T = 0.25
    delta = T / float(periods)
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    z = np.random.normal(0, 1, (n, periods))
    prices_paths = np.zeros((n, periods))
    S1 = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z[:, 0])
    prices_paths[:, 0] = S1
    for i in xrange(1, periods):
        prices_paths[:, i] = prices_paths[:, i - 1] *\
                                np.exp((r - .5 * sigma ** 2) * delta +
                                       sigma * np.sqrt(delta) * z[:, i])
    
    knocin_pos = (prices_paths < H).sum(axis=1) > 0
    prices_in = prices_paths[:, -1] > K
    Ct = np.zeros(n)
    Ct[knocin_pos & prices_in] = 10000
    c1 = discFactor * Ct
    std_origin = c1.std() / np.sqrt(n)
    initial_times = np.apply_along_axis(find_inital_time, 1, prices_paths, H)
    c = get_prices(prices_paths, initial_times, delta, T, K, sigma, r)
    std_reduced = c.std() / np.sqrt(n)
    variance_ratio = (std_origin / std_reduced) ** 2
    return (c.mean(), std_reduced, variance_ratio)
    
    


Ks = [[94, 96], [90, 96], [85, 96], [90, 106]]
Ks = np.array(Ks)
# ix = np.apply_along_axis(lambda x: str(x), 1, Ks)
def f(ls):
    x = []
    for i in ls:
        x.append(str(i))
    return x
ix = f(Ks)
print ix
pricing = np.array([list(digital_option_pricing_bthree(100000, 50,
                                                       i[1], i[0])) for i in Ks])
# print pricing
result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1], "Variance_Ratio": pricing[:, 2]},
                      index=ix)

result  

['[94 96]', '[90 96]', '[85 96]', '[ 90 106]']


,Price,Std,Variance_Ratio
[94 96],3015.336292,5.055817,8.078824
[90 96],424.669998,2.100680,9.014176
[85 96],5.564473,0.098906,54.804588
[ 90 106],13.274360,0.092418,165.337228


Question 5

(a)

If taking 50 steps

In [76]:
n = 100000
def E_option_pricing(n, periods, K, H):
    r = .10
    sigma = .3
    S0 = 100
    T = 0.2
    delta = T / float(periods)
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    z = np.random.normal(0, 1, (n, periods))
    prices_paths = np.zeros((n, periods))
    S1 = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z[:, 0])
    prices_paths[:, 0] = S1
    for i in xrange(1, periods):
        prices_paths[:, i] = prices_paths[:, i - 1] *\
                                np.exp((r - .5 * sigma ** 2) * delta +
                                       sigma * np.sqrt(delta) * z[:, i])
    
    knocin_pos = (prices_paths < H).sum(axis=1) > 0
    Ct = prices_paths[:, -1] - K
    Ct[Ct < 0] = 0
    Ct[~knocin_pos] = 0
    c = discFactor * Ct
    return (c.mean(), c.std() / np.sqrt(n))


Ks = [[95, 100]]
Ks = np.array(Ks)
ix = np.apply_along_axis(lambda x: str(x), 1, Ks)
# print ix
pricing = np.array([list(E_option_pricing(100000, 50,
                                          i[1], i[0])) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]},
                     index=ix)

result  

['[ 95 100]']


,Price,Std
[ 95 100],1.44544,0.01358


If taking 25 steps

In [84]:
pricing = np.array([list(E_option_pricing(100000, 25,
                                          i[1], i[0])) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]},
                     index=ix)

result

,Price,Std
[ 95 100],1.268245,0.012488


(b)

If taking 50 steps

In [28]:
n = 100000


def black_scholes (cp, s, k, t, v, r, div):

        d1 = np.log(s / k) / (v * np.sqrt(t)) + (r + (v ** 2) / 2) * np.sqrt(t) / v
        d2 = d1 - v * np.sqrt(t)
        pv = np.exp(-r * t)
        optprice = s * norm.cdf(d1) - k * pv * norm.cdf(d2)
        return optprice

def E_option_helper(K, St, r, tao, sigma):
    K = float(K)
    Ctao = black_scholes(1, St, K, tao, sigma, r, 0)
    return Ctao

def find_inital_time(price_path, H):
    judge = np.where(price_path < H)[0]
    if len(judge) > 0:
        return judge[0]
    else:
        return -1


def E_get_prices(sample_matrix, inital_time, delta, T, K, sigma, r):
    paras = []
    c = np.zeros(len(inital_time))

    for i in xrange(len(inital_time)):
        if inital_time[i] != -1:
            t = (inital_time[i] + 1) * delta
            tao = T - t
            if tao == 0:
                c[i] = 0
            else:
                St = sample_matrix[i, inital_time[i]]
                discfactor = np.exp(-r * t)
                c[i] = E_option_helper(K, St, r, tao, sigma)
#                 print c[i]
                if c[i] < 0:
                    print (K, St, r, tao, sigma)
    return c
    
        
def E_option_pricing(n, periods, K, H):
    r = .10
    sigma = .3
    S0 = 100
    T = 0.2
    delta = T / float(periods)
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    z = np.random.normal(0, 1, (n, periods))
    prices_paths = np.zeros((n, periods))
    S1 = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z[:, 0])
    prices_paths[:, 0] = S1
    for i in xrange(1, periods):
        prices_paths[:, i] = prices_paths[:, i - 1] *\
                                np.exp((r - .5 * sigma ** 2) * delta +
                                       sigma * np.sqrt(delta) * z[:, i])
    initial_times = np.apply_along_axis(find_inital_time, 1, prices_paths, H)
    c = E_get_prices(prices_paths, initial_times, delta, T, K, sigma, r)
    return (c.mean(), c.std() / np.sqrt(n))


Ks = [[95, 100]]
# ix = np.apply_along_axis(lambda x: str(x), 0, Ks)
def f(ls):
    x = []
    for i in ls:
        x.append(str(i))
    return x
ix = f(Ks)
# print ix
pricing = np.array([list(E_option_pricing(100000, 50,
                                          i[1], i[0])) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]}, index=ix)

result  

,Price,Std
"[95, 100]",1.449312,0.004249


If taking 25 steps

In [29]:
Ks = [[95, 100]]
ix = f(Ks)
# ix = np.apply_along_axis(lambda x: str(x), 0, Ks)
pricing = np.array([list(E_option_pricing(100000, 25,
                                          i[1], i[0])) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]}, index=ix)


result  

,Price,Std
"[95, 100]",1.269575,0.004026


(c)

If taking 25 steps

In [26]:
n = 100000


def black_scholes (cp, s, k, t, v, r, div):

        d1 = np.log(s / k) / (v * np.sqrt(t)) + (r + (v ** 2) / 2) * np.sqrt(t) / v
        d2 = d1 - v * np.sqrt(t)
        pv = np.exp(-r * t)
        optprice = s * norm.cdf(d1) - k * pv * norm.cdf(d2)
        return optprice

def E_option_helper(K, St, r, tao, sigma):
    K = float(K)
    Ctao = black_scholes(1, St, K, tao, sigma, r, 0)
    return Ctao

def find_inital_time(price_path, H):
    judge = np.where(price_path < H)[0]
    if len(judge) > 0:
        return judge[0]
    else:
        return -1


def E_get_prices(sample_matrix, inital_time, delta, T, K, sigma, r):
    paras = []
    c = np.zeros(len(inital_time))

    for i in xrange(len(inital_time)):
        if inital_time[i] != -1:
            t = (inital_time[i] + 1) * delta
            tao = T - t
            if tao == 0:
                c[i] = 0
            else:
                St = sample_matrix[i, inital_time[i]]
                discfactor = np.exp(-r * t)
                c[i] = E_option_helper(K, St, r, tao, sigma)
    return c
    
    
def adjustment_factor(inital_time, theta, z_n):
    paras = []
    adj_factor = np.zeros(len(inital_time))
    adj = norm.pdf(z_n) / norm.pdf(z_n - theta)
    for i in xrange(len(inital_time)):
        if inital_time[i] != -1:
            path = adj[i, :inital_time[i] + 1]
            adj_factor[i] = np.multiply.reduce(path)
    return adj_factor
    
        
def E_option_pricing_importance(n, periods, K, H, theta):
    r = .10
    sigma = .3
    S0 = 100
    T = 0.2
    delta = T / float(periods)
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    z_n = np.random.normal(0, 1, (n, periods))
    z = z_n + theta
    prices_paths = np.zeros((n, periods))
    S1 = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z[:, 0])
    prices_paths[:, 0] = S1
    for i in xrange(1, periods):
        prices_paths[:, i] = prices_paths[:, i - 1] *\
                                np.exp((r - .5 * sigma ** 2) * delta +
                                       sigma * np.sqrt(delta) * z[:, i])
    initial_times = np.apply_along_axis(find_inital_time, 1, prices_paths, H)
    adj = adjustment_factor(initial_times, theta, z)
#     print adj
    c = E_get_prices(prices_paths, initial_times, delta, T, K, sigma, r) * adj
#     print adj
    
    return (c.mean(), c.std() / np.sqrt(n))


Ks = [[95, 100]]
ix = f(Ks)
# ix = np.apply_along_axis(lambda x: str(x), 0, Ks)
pricing = np.array([list(E_option_pricing_importance(100000, 25,
                                                     i[1], i[0], -.45)) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]}, index=ix)

result

[ 0.33053976  0.4406696   0.6381157  ...,  0.51971041  0.51856365
  0.92932574]


,Price,Std
"[95, 100]",1.271532,0.001721


If taking 50 steps

In [25]:
n = 100000


def black_scholes (cp, s, k, t, v, r, div):

    d1 = np.log(s / k) / (v * np.sqrt(t)) + (r + (v ** 2) / 2) * np.sqrt(t) / v
    d2 = d1 - v * np.sqrt(t)
    pv = np.exp(-r * t)
    optprice = s * norm.cdf(d1) - k * pv * norm.cdf(d2)
    return optprice

def E_option_helper(K, St, r, tao, sigma):
    K = float(K)
    Ctao = black_scholes(1, St, K, tao, sigma, r, 0)
    return Ctao

def find_inital_time(price_path, H):
    judge = np.where(price_path < H)[0]
    if len(judge) > 0:
        return judge[0]
    else:
        return -1


def E_get_prices(sample_matrix, inital_time, delta, T, K, sigma, r):
    paras = []
    c = np.zeros(len(inital_time))

    for i in xrange(len(inital_time)):
        if inital_time[i] != -1:
            t = (inital_time[i] + 1) * delta
            tao = T - t
            if tao == 0:
                c[i] = 0
            else:
                St = sample_matrix[i, inital_time[i]]
                discfactor = np.exp(-r * t)
                c[i] = E_option_helper(K, St, r, tao, sigma)
    return c
    
    
def adjustment_factor(inital_time, theta, z_n):
    paras = []
    adj_factor = np.zeros(len(inital_time))
    adj = norm.pdf(z_n) / norm.pdf(z_n - theta)
#     adj = np.exp(-theta * z_n)
    for i in xrange(len(inital_time)):
        if inital_time[i] != -1:
            path = adj[i, :inital_time[i] + 1]
            adj_factor[i] = np.multiply.reduce(path)
    return adj_factor
    
        
def E_option_pricing_importance(n, periods, K, H, theta):
    r = .10
    sigma = .3
    S0 = 100
    T = 0.2
    delta = T / float(periods)
    discFactor = np.exp(-r * T)
    c = np.zeros(n);
    z_n = np.random.normal(0, 1, (n, periods))
    z = z_n + theta
    prices_paths = np.zeros((n, periods))
    S1 = S0 * np.exp((r - .5 * sigma ** 2) * delta + sigma * np.sqrt(delta) * z[:, 0])
    prices_paths[:, 0] = S1
    for i in xrange(1, periods):
        prices_paths[:, i] = prices_paths[:, i - 1] *\
                                np.exp((r - .5 * sigma ** 2) * delta +
                                       sigma * np.sqrt(delta) * z[:, i])
    initial_times = np.apply_along_axis(find_inital_time, 1, prices_paths, H)
    adj = adjustment_factor(initial_times, theta, z)
#     print adj
    c = E_get_prices(prices_paths, initial_times, delta, T, K, sigma, r) * adj
#     print adj
    
    return (c.mean(), c.std() / np.sqrt(n))


Ks = [[95, 100]]
ix = f(Ks)
# ix = np.apply_along_axis(lambda x: str(x), 0, Ks)
pricing = np.array([list(E_option_pricing_importance(100000, 50, i[1],
                                                     i[0], -.45)) for i in Ks])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]}, index=ix)

result

,Price,Std
"[95, 100]",1.445939,0.003093
